In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [10]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [11]:
def scrape_links(page_number):
    url=f"https://www.beritajakarta.id/news_index/page/{page_number}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('div',{"class": "media blog-list news-index"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [12]:
for page_number in range(1, jumlah_index * 10, 10):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 10 links from page 1
Total Links: 10


In [13]:
def scrape_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
                
            elem = soup.find('div', {"class": "text-secondary"})
            
            if elem:
                input_string = elem.text
                parts = input_string.split("Reporter :")
                date_text = parts[0].strip()
                editor_and_view = parts[1].split("Editor :")
                author_text = editor_and_view[0].strip()
                remaining_text = editor_and_view[1]
                editor_parts = remaining_text.split()
                editor_text = editor_parts[0]
                viewer_text = editor_parts[-1]
            else:
                author_text = "Author not found"
                date_text = "Date not found"
                editor_text = "Editor not found"
                viewer_text = "Viewer not found"
                
                
            # Author berita
            # author_elem = soup.find('div', {"class": "jeg_meta_author"})
            # if author_elem:
            #     author_text = author_elem.find('a')
            #     author_text = author_text.text
            # else:
            #     author_text = "Author not found"
                     
            # tanggal berita
            # date_elem = soup.find('div', {"class": "jeg_meta_date"})
            # if date_elem:
            #     date_text = date_elem.find('a')
            #     date_text = date_text.text
            # else:
            #     date_text = "Date not found"
            # Category berita
            category_elements = soup.find_all('li', {"class":"breadcrumb-item"}) 
            if category_elements:
                category_text= category_elements[1].text
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "news-content"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'editor' : editor_text,
                            'view' : viewer_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'jakarta',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [14]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [15]:
df = pd.DataFrame(results)
print("Total Hasil ",len(results))
df.head(10)

Total Hasil  10


,title,author,category,editor,view,date,content,region,link
0,Heru Resmikan Bentara Budaya Art Gallery di Me...,Aldi Geri Lumban Tobing,Seni & Budaya,Erikyanri,205,"Selasa, 26 September 2023 13:59","Penjabat (Pj) Gubernur Provinsi DKI Jakarta, H...",jakarta,https://www.beritajakarta.id/read/129102/heru-...
1,Delapan Pohon Rawan Tumbang di Cengkareng Dipa...,TP Moan Simanjuntak,Lintas Kota,Erikyanri,115,"Selasa, 26 September 2023 15:24",Suku Dinas Pertamanan dan Hutan Kota (Sudin Ta...,jakarta,https://www.beritajakarta.id/read/129104/delap...
2,Pemkot Jaktim Terima 500 Pohon dari CSR,Nurito,Jakarta Hari Ini,Budhy,1603,"Selasa, 26 September 2023 11:52","Pemerintah Kota Jakarta Timur , Selasa (26/9),...",jakarta,https://www.beritajakarta.id/read/129093/pemko...
3,Turap Saluran Jl Pelepah Asri 1 Ditinggikan,Budhi Firmansyah Surapati,Lintas Kota,Budhy,326,"Selasa, 26 September 2023 09:49",Delapan personel Satgas Sumber Daya Air (SDA) ...,jakarta,https://www.beritajakarta.id/read/129090/turap...
4,Ini Langkah Pemkot Jaktim Antisipasi Dampak K...,Nurito,Jakarta Hari Ini,Budhy,122,"Selasa, 26 September 2023 14:56","Pemerintah Kota (Pemkot) Jakarta Timur, mengkl...",jakarta,https://www.beritajakarta.id/read/129108/ini-l...
5,Penghuni eks Kampung Bayam Sepakat Direlokasi ...,Budhi Firmansyah Surapati,Jakarta Hari Ini,Budhy,292,"Selasa, 26 September 2023 12:13",Penghuni eks Kampung Bayam yang bertahan dalam...,jakarta,https://www.beritajakarta.id/read/129096/pengh...
6,Ditemukan 10 Titik Sumber Mata Air di Area Wad...,Nurito,Jakarta Hari Ini,Budhy,324,"Selasa, 26 September 2023 12:43",Sebanyak 10 titik sumber mata air ditemukan di...,jakarta,https://www.beritajakarta.id/read/129106/ditem...
7,Penataan Kawasan Unggulan di Karang Anyar Ramp...,Wuri Setyaningsih,Lintas Kota,Andry,93,"Selasa, 26 September 2023 15:39",Penataan kawasan unggulan triwulan ketiga di J...,jakarta,https://www.beritajakarta.id/read/129091/penat...
8,KONI DKI dan BPJS Ketenagakerjaan Teken Kerja ...,Folmer,Olahraga,Erikyanri,92,"Selasa, 26 September 2023 15:41",KONI DKI Jakarta bersama BPJS Ketenagakerjaan ...,jakarta,https://www.beritajakarta.id/read/129119/koni-...
9,Penataan Kawasan di Jalan Sa'bah Capai 95 Persen,Tiyo Surya Sakti,Pembangunan,Andry,391,"Selasa, 26 September 2023 09:03",Penataan kawasan unggulan triwulan ketiga di p...,jakarta,https://www.beritajakarta.id/read/129079/penat...


save data

In [16]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/beritaJakarta_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ./excel/beritaJakarta_2023-09-26_16-42-19.xlsx
